<img src="./Assets/1.1 AgWorkHeaderImage.png">

## Tool Use

It combines MEMORY, REASONING, PLANNING, and __ACTIONS (often via EXTERNAL TOOLS)__ to fulfil TASKS.

Tools are methods that help LLMs accomplish actions that require interaction with entities outside of the model.

<img src="./Assets/2.1 Tool Use Single.png">

---

In [76]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [77]:
from crewai import LLM
llm = LLM(model="gpt-4o-mini")

In [78]:
from crewai import Agent
from crewai_tools import SerperDevTool

In [79]:
Query_Responder=Agent(
    role=f"""AI Web Search Assistant""",
    goal=f"Help users by either answering their queries directly using built-in knowledge or performing a live web search using the Google Serper API when up-to-date or external information is required.",
    backstory=f"""You were built to bridge the gap between real-time information and artificial intelligence. Equipped with access to the web through the Serper API, you excel at retrieving the most recent, relevant data when needed. But you also know when to keep things simple — using your own understanding to answer straightforward questions without delay. You’re the perfect blend of real-time access and inherent intelligence.""",
    tools=[SerperDevTool()],
    verbose=True,
    llm=llm
)


In [80]:
from crewai import Task

In [81]:
handle_user_query= Task(
    description=(
        "Receive and respond to a user's question - {user_input} "
        "If the query is simple or general knowledge (e.g., definitions, common facts), "
        "answer using your own knowledge. "
        "If the query appears to require current or web-based information "
        "(e.g., news, trending topics, recent data), "
        "perform a web search using the Serper API tool to gather relevant information first. "
        "Ensure your response is concise, helpful, and clearly indicates whether it was based on a search or internal understanding."
    ),
    expected_output=(
        "A final answer to the user's question, either using internal knowledge or based on live search results. "
        "If using search, summarize the result clearly and include key information retrieved. "
        "Indicate whether the result was from the web or internal knowledge."
    ),
    agent=Query_Responder
)

In [82]:
from crewai import Crew, Process

crew = Crew(
    agents=[Query_Responder],
    tasks=[handle_user_query],
    process=Process.sequential
)


In [89]:
result = crew.kickoff(inputs={"user_input": "Who won the first t20 world cup?"})

# Agent: AI Web Search Assistant
## Task: Receive and respond to a user's question - Who won the first t20 world cup? If the query is simple or general knowledge (e.g., definitions, common facts), answer using your own knowledge. If the query appears to require current or web-based information (e.g., news, trending topics, recent data), perform a web search using the Serper API tool to gather relevant information first. Ensure your response is concise, helpful, and clearly indicates whether it was based on a search or internal understanding.


# Agent: AI Web Search Assistant
## Final Answer: 
The first T20 World Cup was held in 2007, and it was won by India, who defeated Pakistan in the final.




In [90]:
from IPython.display import Markdown, display
display(Markdown(result.raw))

The first T20 World Cup was held in 2007, and it was won by India, who defeated Pakistan in the final.

---

In [91]:
Token=os.environ["ASTRA_TOKEN"]
EndPoint=os.environ["ASTRA_ENDPOINT"]

In [92]:
from astrapy import DataAPIClient

# Initialize the client
client = DataAPIClient(Token)
db = client.get_database_by_api_endpoint(
  EndPoint
)

print(f"Connected to Astra DB: {db.list_collection_names()}")

Connected to Astra DB: ['CollectionSmallEmbeddings']


In [93]:
collection = db.get_collection("CollectionSmallEmbeddings")

In [94]:
query="How many days of paternity leave can be availed?"

In [95]:
from openai import OpenAI
client = OpenAI()

query_embedding=client.embeddings.create(
  model="text-embedding-3-small",
  input=query,
  encoding_format="float"
)


In [96]:
from astrapy.data_types import DataAPIVector

results = collection.find(
    sort={"$vector": DataAPIVector(query_embedding.data[0].embedding)},
    limit=5,
    include_similarity=True
)


In [97]:
for doc in results:
    print(doc)


{'_id': '693147c55d554642bab1ed63f2285c55', 'page_content': 'childbirth.\n \nEligible\n \nemployees\n \nare\n \nentitled\n \nto\n \nup\n \nto\n \n7\n \ndays\n \nof\n \npaid\n \npaternity\n \nleave,\n \nwhich\n \ncan\n \nbe\n \nused\n \nwithin\n \n6\n \nmonths\n \nof\n \nthe\n \nchild’s\n \nbirth.\n ○  Employees  are  required  to  provide  proof  of  the  birth  (e.g.,  a  birth  certificate  or  \nhospital\n \nrecords)\n \nwhen\n \napplying\n \nfor\n \npaternity\n \nleave.\n \n \nf.  Long-Term  Leave  for  Medical  Conditions  or  Personal  Circumstances  \n1.  Leave  for  Long-Term  Medical  Conditions:  \n ○  Employees  dealing  with  long-term  medical  conditions  that  require  extended  time  \noff\n \nmay\n \nrequest\n \nlong-term\n \nmedical\n \nleave\n.\n \nThis\n \nleave\n \nis\n \ntypically\n \nunpaid\n \nand\n \nrequires\n \nmedical\n \ndocumentation\n \nto\n \nsubstantiate\n \nthe\n \nrequest.\n ○  The  company  will  review  each  case  individually  and  may  approve  l

In [98]:
from astrapy import DataAPIClient
from astrapy.data_types import DataAPIVector
from openai import OpenAI

def search_documents(query: str, top_k: int = 5) -> list[str]:
    # Set up OpenAI
    openai_client = OpenAI()
    query_embedding = openai_client.embeddings.create(
        model="text-embedding-3-small",
        input=query,
        encoding_format="float"
    ).data[0].embedding

    # Connect to Astra
    astra_client = DataAPIClient(Token)
    db = astra_client.get_database_by_api_endpoint(EndPoint)
    collection = db.get_collection("CollectionSmallEmbeddings")

    # Query Astra
    cursor = collection.find(
        sort={"$vector": DataAPIVector(query_embedding)},
        limit=top_k,
        include_similarity=True
    )

    # Return documents
    results = []
    for doc in cursor:
        text = doc.get("text") or str(doc)
        score = doc.get("$similarity", 0)
        results.append(f"[{score:.4f}] {text}")
    return results


In [99]:
from crewai.tools import BaseTool

class AstraSearchTool(BaseTool):
    name: str = "VectorTool"
    description: str = "Search the vector store for information about employee leave policy of AKAIWorks"

    def _run(self, query: str) -> str:
        # This is where your vector search logic goes
        results = search_documents(query)
        return "\n".join(results)


In [100]:
astra_tool = AstraSearchTool()

In [101]:
llm = LLM(
    model="openai/gpt-4o-mini", # call model by provider/model_name
    temperature=0.8,
    max_tokens=150,
    top_p=0.9,
    frequency_penalty=0.1,
    presence_penalty=0.1,
    stop=["END"],
    seed=42
)


In [116]:
Query_Responder=Agent(
    role=f"""AI Search Assistant""",
    goal=f"Help users by either answering their queries directly using built-in knowledge, or use the VectorTool when the query is about employee leave policy of AKAIWorks or performing a live web search using the Google Serper API when up-to-date or external information is required.",
    backstory=f"""You were built to bridge the gap between real-time information and artificial intelligence. Equipped with access to the web through the Serper API, employee leave policy via the vector tool, you excel at retrieving the most recent, relevant data when needed. But you also know when to keep things simple — using your own understanding to answer straightforward questions without delay. You’re the perfect blend of real-time access and inherent intelligence. Remember that company name might not be explicitly mentioned but assume that the leave policy query is implicitly for the organisation.""",
    tools=[SerperDevTool(),astra_tool],
    verbose=True,
    llm=llm
)


In [127]:
handle_user_query= Task(
    description=(
        "Receive and respond to a user's question - {user_input} "
        "If the query is simple or general knowledge (e.g., definitions, common facts), "
        "answer using your own knowledge. "
        "If the query appears to require current or web-based information "
        "(e.g., news, trending topics, recent data), "
        "perform a web search using the Serper API tool to gather relevant information first. "
        "If the query is about the employee leave policy of AKAIWorks, "
        "perform a vector database search using vector search tool."
        "Remember that company name might not be explicitly mentioned but assume that the leave policy query is implicitly for the organisation."
        "Ensure your response is concise, helpful, and clearly indicates whether it was based on a search or internal understanding."
        "Always end your answer by saying which source you used: [LLM], [Web Search], or [Vector DB]."
    ),
    expected_output=(
        "A final answer to the user's question, either using internal knowledge or based on live search results. "
        "If using search, summarize the result clearly and include key information retrieved. "
        "Indicate whether the result was from the web or internal knowledge."
        "Always end your answer by saying which source you used: [LLM], [Web Search], or [Vector DB]."
    ),
    agent=Query_Responder
)

In [128]:
crew = Crew(
    agents=[Query_Responder],
    tasks=[handle_user_query],
    process=Process.sequential
)


In [129]:
result = crew.kickoff(inputs={"user_input": "How many paternity leaves are employees typically entitled to in a year in the United States?"})

# Agent: AI Search Assistant
## Task: Receive and respond to a user's question - How many paternity leaves are employees typically entitled to in a year in the United States? If the query is simple or general knowledge (e.g., definitions, common facts), answer using your own knowledge. If the query appears to require current or web-based information (e.g., news, trending topics, recent data), perform a web search using the Serper API tool to gather relevant information first. If the query is about the employee leave policy of AKAIWorks, perform a vector database search using vector search tool.Remember that company name might not be explicitly mentioned but assume that the leave policy query is implicitly for the organisation.Ensure your response is concise, helpful, and clearly indicates whether it was based on a search or internal understanding.Always end your answer by saying which source you used: [LLM], [Web Search], or [Vector DB].


# Agent: AI Search Assistant
## Final Answer: 

In [130]:
display(Markdown(result.raw))


In the United States, there is no federal law mandating paid paternity leave. However, many employers offer paternity leave as part of their benefits package, which can range from a few days to several weeks. Typically, employees may be entitled to around 1 to 2 weeks of paid paternity leave, but this can vary significantly by company policy and state laws. Some states have implemented their own family leave laws that may provide for longer periods of leave. [LLM]

---